In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold
import pickle

import os
import sys
sys.path.append('/home/cggyoshikawa/work/kaggle/OPVP')


In [28]:
val_columns = ['log_return_realized_volatility',
 'log_return_mean',
 'log_return_std',
 'log_return_sum',
 'log_return2_realized_volatility',
 'log_return2_mean',
 'log_return2_std',
 'log_return2_sum',
 'wap_balance_mean',
 'wap_balance_std',
 'wap_balance_sum',
 'price_spread_mean',
 'price_spread_std',
 'price_spread_sum',
 'bid_spread_mean',
 'bid_spread_std',
 'bid_spread_sum',
 'ask_spread_mean',
 'ask_spread_std',
 'ask_spread_sum',
 'volume_imbalance_mean',
 'volume_imbalance_std',
 'volume_imbalance_sum',
 'total_volume_mean',
 'total_volume_std',
 'total_volume_sum',
 'wap_mean',
 'wap_std',
 'wap_sum',
 'log_return_realized_volatility_450',
 'log_return_mean_450',
 'log_return_std_450',
 'log_return_sum_450',
 'log_return2_realized_volatility_450',
 'log_return2_mean_450',
 'log_return2_std_450',
 'log_return2_sum_450',
 'wap_balance_mean_450',
 'wap_balance_std_450',
 'wap_balance_sum_450',
 'price_spread_mean_450',
 'price_spread_std_450',
 'price_spread_sum_450',
 'bid_spread_mean_450',
 'bid_spread_std_450',
 'bid_spread_sum_450',
 'ask_spread_mean_450',
 'ask_spread_std_450',
 'ask_spread_sum_450',
 'volume_imbalance_mean_450',
 'volume_imbalance_std_450',
 'volume_imbalance_sum_450',
 'total_volume_mean_450',
 'total_volume_std_450',
 'total_volume_sum_450',
 'wap_mean_450',
 'wap_std_450',
 'wap_sum_450',
 'log_return_realized_volatility_300',
 'log_return_mean_300',
 'log_return_std_300',
 'log_return_sum_300',
 'log_return2_realized_volatility_300',
 'log_return2_mean_300',
 'log_return2_std_300',
 'log_return2_sum_300',
 'wap_balance_mean_300',
 'wap_balance_std_300',
 'wap_balance_sum_300',
 'price_spread_mean_300',
 'price_spread_std_300',
 'price_spread_sum_300',
 'bid_spread_mean_300',
 'bid_spread_std_300',
 'bid_spread_sum_300',
 'ask_spread_mean_300',
 'ask_spread_std_300',
 'ask_spread_sum_300',
 'volume_imbalance_mean_300',
 'volume_imbalance_std_300',
 'volume_imbalance_sum_300',
 'total_volume_mean_300',
 'total_volume_std_300',
 'total_volume_sum_300',
 'wap_mean_300',
 'wap_std_300',
 'wap_sum_300',
 'log_return_realized_volatility_150',
 'log_return_mean_150',
 'log_return_std_150',
 'log_return_sum_150',
 'log_return2_realized_volatility_150',
 'log_return2_mean_150',
 'log_return2_std_150',
 'log_return2_sum_150',
 'wap_balance_mean_150',
 'wap_balance_std_150',
 'wap_balance_sum_150',
 'price_spread_mean_150',
 'price_spread_std_150',
 'price_spread_sum_150',
 'bid_spread_mean_150',
 'bid_spread_std_150',
 'bid_spread_sum_150',
 'ask_spread_mean_150',
 'ask_spread_std_150',
 'ask_spread_sum_150',
 'volume_imbalance_mean_150',
 'volume_imbalance_std_150',
 'volume_imbalance_sum_150',
 'total_volume_mean_150',
 'total_volume_std_150',
 'total_volume_sum_150',
 'wap_mean_150',
 'wap_std_150',
 'wap_sum_150',
 'trade_log_return_realized_volatility',
 'trade_seconds_in_bucket_count_unique',
 'trade_size_sum',
 'trade_order_count_mean',
 'trade_log_return_realized_volatility_450',
 'trade_seconds_in_bucket_count_unique_450',
 'trade_size_sum_450',
 'trade_order_count_mean_450',
 'trade_log_return_realized_volatility_300',
 'trade_seconds_in_bucket_count_unique_300',
 'trade_size_sum_300',
 'trade_order_count_mean_300',
 'trade_log_return_realized_volatility_150',
 'trade_seconds_in_bucket_count_unique_150',
 'trade_size_sum_150',
 'trade_order_count_mean_150',
 'stock_id',
 'stock_id_target_enc']

In [29]:
data_dir = '../input/optiver-realized-volatility-prediction/'

In [30]:
df_test = pd.read_pickle('/home/yoshikawa/work/kaggle/OPVP/output/feature/nb001/test.pkl')

y_pred = df_test[['row_id']]
X_test = df_test.drop(['time_id', 'row_id'], axis=1)
X_test

,stock_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id_target_enc
0,0,0.000294,0.000147,0.000208,0.000294,0.000252,0.000139,0.000159,0.000277,0.000145,0.000022,0.000436,0.000557,0.000028,0.001671,0.000393,0.0,0.00118,-0.000115,0.000113,-0.000344,164.666667,165.711597,494.0,350.666667,65.271229,1052.0,1.000405,0.00017,3.001215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,3.0,201.0,3.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [31]:
target = np.zeros(len(X_test))
models = []
for i in range(5):
    with open('../output/model/nb001/lgbm'+str(i)+'.pkl', 'rb') as f:
        models.append(pickle.load(f))

for model in models:
    pred = model.predict(X_test[val_columns], num_iteration=model.best_iteration)
    target += pred / len(models)
    
y_pred = y_pred.assign(target=target)
y_pred

,row_id,target
0,0-4,0.001399
1,0-32,0.001412
2,0-34,0.001412


In [32]:
# y_pred.to_csv('submission.csv',index = False)